# 

In [68]:
import pandas as pd
import sourmash
import glob
import os
import re

In [69]:
fmg_csv = "../output.roux2017/fastmultigather/roux2017-x-refseq-v219.dna.k21-sc10.t0.gather.csv"

## Load gather csvs with multiple parameters at once

In [75]:
fmg_csvs = glob.glob("../output.roux2017/fastmultigather/roux2017-x-refseq-v219.dna.k*-sc*.t0.gather.csv")

def extract_parameters_from_filename(filename):
    match = re.search(r'k(\d+)-sc(\d+).t(\d+)', filename)
    if match:
        ksize = int(match.group(1))
        scaled = int(match.group(2))
        threshold = int(match.group(3))
        return ksize, scaled, threshold
    else:
        return None, None, None

In [76]:
fmg_dfs = []
for inf in fmg_csvs:
    ksize, scaled, threshold = extract_parameters_from_filename(inf)
    inD = pd.read_csv(inf)
    inD['ksize'] = ksize
    inD['scaled'] = scaled
    inD['threshold'] = threshold
    fmg_dfs.append(inD)

fmg = pd.concat(fmg_dfs)

In [77]:
fmg.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,ksize,scaled,threshold
0,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|479336462|ref|NC_021061.1| Mycobacterium ph...,fe4925d714f82bb52c847ac7ec487b1d,1.000000,43700,21,100,0
1,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|273809542|ref|NC_013597.1| Aggregatibacter ...,7eb3626e5efc71a3a6a4f0310f3f1113,0.975446,43700,21,100,0
2,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|1070619425|ref|NC_031241.1| Staphylococcus ...,74bc58e1c10927dc5e6c4eb92f5b72f9,1.000000,42200,21,100,0
3,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|157311135|ref|NC_009819.1| Streptococcus ph...,f0c5b5931b0c7d993b4e2ec278df773f,1.000000,41300,21,100,0
4,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|197935853|ref|NC_011201.1| Ralstonia phage ...,a023a5fa98a68ce2d90884efcad473e7,1.000000,41100,21,100,0


## Read in the Sample compositions tsvs

In [8]:
samplecomp_tsvs = glob.glob('../roux2017-samplecomp/*tsv')
samplecomp_tsvs

['../roux2017-samplecomp/Sample_10_comp.tsv',
 '../roux2017-samplecomp/Sample_11_comp.tsv',
 '../roux2017-samplecomp/Sample_7_comp.tsv',
 '../roux2017-samplecomp/Sample_6_comp.tsv',
 '../roux2017-samplecomp/Sample_1_comp.tsv',
 '../roux2017-samplecomp/Sample_4_comp.tsv',
 '../roux2017-samplecomp/Sample_5_comp.tsv',
 '../roux2017-samplecomp/Sample_13_comp.tsv',
 '../roux2017-samplecomp/Sample_12_comp.tsv',
 '../roux2017-samplecomp/Sample_3_comp.tsv',
 '../roux2017-samplecomp/Sample_2_comp.tsv',
 '../roux2017-samplecomp/Sample_14_comp.tsv',
 '../roux2017-samplecomp/Sample_9_comp.tsv',
 '../roux2017-samplecomp/Sample_8_comp.tsv']

In [33]:
cmpDF = pd.concat((pd.read_csv(f, sep='\t').assign(Sample=os.path.basename(f).split('_comp.tsv')[0]) for f in samplecomp_tsvs), ignore_index=True)
cmpDF.rename(columns={'## Virus': 'Virus'}, inplace=True)
cmpDF.head()

,Virus,Name,Coverage,Sample
0,22855216,Pseudomonas phage phi-12 segment S,0.058544,Sample_10
1,134288688,Burkholderia phage phiE12-2 chromosome,0.054693,Sample_10
2,530787156,Bacillus phage Troll,0.051090,Sample_10
3,269838908,Morganella phage MmP1,0.047719,Sample_10
4,557307743,Mycobacterium phage Fredward,0.044565,Sample_10


In [64]:
fmg['Name'] = fmg['match_name'].str.split(' ', n=1, expand=True)[1]
fmg['gi'] = fmg['match_name'].str.split('|', expand=True)[1]
fmg.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,Name,gi
0,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|388684650|ref|NC_017972.1| Pseudomonas phag...,8e4254b5af76d576145006347d533040,0.999964,281400,"Pseudomonas phage Lu11, complete genome",388684650
1,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|414088861|ref|NC_019408.1| Caulobacter phag...,e1376543b913375166cbb1750b40e2e4,1.000000,213260,"Caulobacter phage CcrRogue, complete genome",414088861
2,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|418487144|ref|NC_019443.1| Synechococcus ph...,f9f22f4efe6f9e4aa35902ba4dd5b534,0.999710,172550,"Synechococcus phage metaG-MbCM1, complete genome",418487144
3,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|460042674|ref|NC_020486.1| Synechococcus ph...,f5d4f2e624ffb0caed5a7abe11c536df,0.995892,170300,"Synechococcus phage S-RIM8 A.HR1, complete genome",460042674
4,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|29366675|ref|NC_000866.4| Enterobacteria ph...,f7691cb10a71235d9ef18cfab0b9a642,0.999701,167200,"Enterobacteria phage T4, complete genome",29366675


## start with a single sample (sample 5)

In [49]:
sample5_fmg = fmg[fmg['query_name'] == "Sample_5"]
sample5_fmg.shape

(457, 7)

In [52]:
sample5_fmg.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,Name
0,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|388684650|ref|NC_017972.1| Pseudomonas phag...,8e4254b5af76d576145006347d533040,0.999964,281400,"Pseudomonas phage Lu11, complete genome"
1,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|414088861|ref|NC_019408.1| Caulobacter phag...,e1376543b913375166cbb1750b40e2e4,1.000000,213260,"Caulobacter phage CcrRogue, complete genome"
2,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|418487144|ref|NC_019443.1| Synechococcus ph...,f9f22f4efe6f9e4aa35902ba4dd5b534,0.999710,172550,"Synechococcus phage metaG-MbCM1, complete genome"
3,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|460042674|ref|NC_020486.1| Synechococcus ph...,f5d4f2e624ffb0caed5a7abe11c536df,0.995892,170300,"Synechococcus phage S-RIM8 A.HR1, complete genome"
4,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|29366675|ref|NC_000866.4| Enterobacteria ph...,f7691cb10a71235d9ef18cfab0b9a642,0.999701,167200,"Enterobacteria phage T4, complete genome"


In [51]:
sample5_cmp = cmpDF[cmpDF['Sample'] == 'Sample_5']
sample5_cmp.shape

(508, 4)

In [54]:
sample5_cmp.head()

,Virus,Name,Coverage,Sample
4581,31044225,Enterobacteria phage BP-4795,0.093009,Sample_5
4582,725915949,Staphylococcus phage phiSa119,0.084515,Sample_5
4583,431810540,Helicobacter phage KHP30 DNA,0.076783,Sample_5
4584,9632893,Streptococcus phage Sfi19,0.069745,Sample_5
4585,744692834,Salmonella phage LSPA1,0.063340,Sample_5


In [58]:
sample5_fmg[sample5_fmg['Name'].str.contains('Enterobacteria')].head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,Name
4,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|29366675|ref|NC_000866.4| Enterobacteria ph...,f7691cb10a71235d9ef18cfab0b9a642,0.999701,167200,"Enterobacteria phage T4, complete genome"
20,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|31044225|ref|NC_004813.1| Enterobacteria ph...,fbf18db4b87ed8b83a3912d1e88cf7b3,1.000000,57910,"Enterobacteria phage BP-4795, complete genome"
58,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|89888607|ref|NC_007817.1| Enterobacteria ph...,48498256fd4f45e4162266d34b89f453,0.059034,850,"Enterobacteria phage ID2 Moscow/ID/2001, compl..."
83,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|422934269|ref|NC_019501.1| Enterobacteria p...,e3a257498956e99ae2b98f0b9a706ba2,0.981541,40630,"Enterobacteria phage IME10, complete genome"
129,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|428765613|ref|NC_019705.1| Enterobacteria p...,5e0bac0bee7f3b6b21772f9f6730781b,0.851643,38960,"Enterobacteria phage mEpX2, complete genome"


In [57]:
fmg['match_name'].nunique()

2651

## Merge based on gi number match